# ECG Split 1D-CNN Server Side
This code is the server part of ECG split 1D-CNN model for **single** client and a server.

## Import required packages, connect to Duet

In [1]:
from pathlib import Path
import numpy as np
import h5py
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD
torch.set_default_tensor_type('torch.cuda.FloatTensor')

import syft as sy

project_path = Path.cwd().parent
print(f'project_path: {project_path}')

# duet = sy.join_duet(loopback=True)
duet = sy.join_duet(target_id='332b215d80ccbd33aeedfc2ce640ba00')

project_path: /mnt/batch/tasks/shared/LS_root/mounts/clusters/splitlearningcomputer/code/Users/dkn.work/split-learning-1D-HE
🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 014ec143ef1ea473f0643d7a0bcb03a4

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


## Check the pointers to the data

In [2]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 21617bfc219b45b2a515d96d486b9741>,[x_train],training input data,<class 'torch.Tensor'>
1,<UID: 8d5ca6451cfd44f28d08d509da157840>,[y_train],training output data,<class 'torch.Tensor'>
2,<UID: 67a45a6773764807b0fa32b4c9d58e9e>,[x_test],testing input data,<class 'torch.Tensor'>
3,<UID: cee66997a38c41899b3287a31ea1439b>,[y_test],testing output data,<class 'torch.Tensor'>


In [3]:
x_train_ptr = duet.store["x_train"]
y_train_ptr = duet.store["y_train"]
x_test_ptr = duet.store["x_test"]
y_test_ptr = duet.store["y_test"]

After the user uses  `duet.requests.add_handler(action="accept")` to accept all requests,
the server can just get the data without submitting a request everytime

In [5]:
try:
    print(f'y_train: {y_train_ptr.get_copy()}')
except Exception as e:
    y_train_ptr.request(reason="y_train request")
    print(e)

y_train: tensor([2, 4, 2,  ..., 1, 4, 0], device='cpu')


Get the data loader pointers for remote training

In [6]:
batch_size = 32  # used for training
xdl_ptr = duet.torch.utils.data.DataLoader(x_train_ptr, batch_size=batch_size)
ydl_ptr = duet.torch.utils.data.DataLoader(y_train_ptr, batch_size=batch_size)
xtdl_ptr = duet.torch.utils.data.DataLoader(x_test_ptr, batch_size=batch_size)
ytdl_ptr = duet.torch.utils.data.DataLoader(y_test_ptr, batch_size=batch_size)

In [7]:
xdl_ptr

## Define ECG 1D CNN model for both the client and server sides
Client side has **2 convolution layers**.
Server side has **2 fully connected layers**.


In [7]:
class EcgClient(sy.Module):
    def __init__(self, torch_ref):
        super(EcgClient, self).__init__(torch_ref=torch_ref)
        self.conv1 = self.torch_ref.nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = self.torch_ref.nn.LeakyReLU()
        self.pool1 = self.torch_ref.nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = self.torch_ref.nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = self.torch_ref.nn.LeakyReLU()
        self.pool2 = self.torch_ref.nn.MaxPool1d(2)  # 32 x 16
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 32 * 16)
        return x

class EcgServer(sy.Module):
    def __init__(self, torch_ref):
        super(EcgServer, self).__init__(torch_ref=torch_ref)
        self.linear3 = nn.Linear(32 * 16, 128)
        self.relu3 = nn.LeakyReLU() 
        self.linear4 = nn.Linear(128, 5)
        self.softmax4 = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        x = self.softmax4(x)
        return x

### Set random seed

In [8]:
seed = 69  # the meaning of life
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [9]:
remote_torch = duet.torch
remote_torch.manual_seed(seed)

### Make the models and assign intial weight as same as non-split model

In [18]:
ecg_client = EcgClient(torch_ref=torch)
checkpoint = torch.load("init_weight.pth")
ecg_client.conv1.weight.data = checkpoint["conv1.weight"]
ecg_client.conv1.bias.data = checkpoint["conv1.bias"]
ecg_client.conv2.weight.data = checkpoint["conv2.weight"]
ecg_client.conv2.bias.data = checkpoint["conv2.bias"]

ecg_server = EcgServer(torch_ref=torch)
checkpoint = torch.load("init_weight.pth")
ecg_server.linear3.weight.data = checkpoint["linear3.weight"]
ecg_server.linear3.bias.data = checkpoint["linear3.bias"]
ecg_server.linear4.weight.data = checkpoint["linear4.weight"]
ecg_server.linear4.bias.data = checkpoint["linear4.bias"]

Send the server's model to cuda, the client model's pointer to duet. The client trains on CPU

In [19]:
server_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(f"server's device: {torch.cuda.get_device_name(0)}")
ecg_server.cuda(server_device)

ecg_client_ptr = ecg_client.send(duet)

server's device: Tesla K80


Lets ask the Data Owner if his Machine has CUDA, then put the model on CUDA if he has it

In [12]:
client_cuda = False
client_cuda_ptr = remote_torch.cuda.is_available()
client_cuda = bool(client_cuda_ptr.get(
                    request_block=True,
                    reason="To run test and inference locally",
                    timeout_secs=5,  # change to something slower
))
print(f'client has CUDA? {client_cuda}')

client_device_ptr = remote_torch.device("cuda" if client_cuda else "cpu")
print(f'client device: {client_device_ptr.type.get_copy()}')
# if we have CUDA lets send our model to the GPU
if client_cuda:
    ecg_client_ptr.cuda(client_device_ptr)
else:
    ecg_client_ptr.cpu()

client has CUDA? True
client device: cuda


Check again the initial weights of the created ecg_client and the remote model after sending to duet

In [ ]:
a = ecg_client_ptr.conv1.get_copy()
torch.equal(a.weight.data, ecg_client.conv1.weight.data)

### Set other hyperparameters in the model
Hyperparameters here should be same with the client side.

In [13]:
batch_size = 32
total_batch = 414  # 32*414=13248. We have 13245 data samples

epoch = 400
criterion = nn.CrossEntropyLoss()
lr = 0.001

optim_client = remote_torch.optim.Adam(params=ecg_client_ptr.parameters(), lr=lr)
optim_server = torch.optim.Adam(params=ecg_server.parameters(), lr=lr)

## Training process

In [ ]:
from tqdm import tqdm

train_losses = list()
train_accs = list()
test_losses = list()
test_accs = list()
best_test_acc = 0  # best test accuracy
for e in range(epoch):
    print(f"Epoch {e+1} - train ", end='')
    
    train_loss = 0.0
    correct, total = 0, 0
    for i in tqdm(range(total_batch)):
        # get the next batch's data
        inputs_ptr = next(iter(xdl_ptr))
        gt_ptr = next(iter(ydl_ptr))
        # initialize all gradients to zero
        optim_server.zero_grad()
        optim_client.zero_grad()
        # compute and get the activation signals from the first half of the network
        activations_ptr = ecg_client_ptr(inputs_ptr)
        # the server still gets access to plain activation signals
        activations = activations_ptr.clone().get(request_block=True)
        print(activations.device)
        # the server continues the forward pass on the activation maps
        # activations_server = activations.to(server_device)  # put on GPU
        y_hat = ecg_server(activations)
        # the server asks to access ground truths in plain text, then put on GPU
        y_gt = gt_ptr.get_copy().to(server_device)
        # calculates cross-entropy loss
        loss = criterion(y_hat, y_gt)
        train_loss += loss.item()
        correct += torch.sum(y_hat.argmax(dim=1) == y_gt).item()
        # backward propagation (calculating gradients of the loss w.r.t the weights)
        loss.backward()
        # send the gradients to the client
        client_grad_ptr = activations.grad.clone().send(duet)
        # update the gradients of the client's model
        activations_ptr.backward(client_grad_ptr)
        # update the weights based on the gradients
        optim_client.step()
        optim_server.step()

        total += len(y_gt)

    train_losses.append(train_loss / total_batch)
    train_accs.append(correct / total)

    print(f'loss: {train_losses[-1]: .4f}, accuracy: {train_accs[-1]*100: 2f}')

    # testing
    with torch.no_grad():  
        test_loss = 0.0
        correct, total = 0, 0
        for i in tqdm(range(total_batch)):
            # get the next batch's data
            test_inputs_ptr = next(iter(xtdl_ptr))
            test_gt_ptr = next(iter(ytdl_ptr))
            # forward pass
            activations_ptr = ecg_client_ptr(test_inputs_ptr)
            activations = activations_ptr.clone().get(request_block=True)
            y_hat = ecg_server(activations.to(server_device))
            # the server asks to access ground truths in plain text
            y_gt = test_gt_ptr.get_copy().to(server_device)
            # calculate test loss
            loss = criterion(y_hat, y_gt)
            test_loss += loss.item()
            correct += torch.sum(y_hat.argmax(dim=1) == y_gt).item()
            total += len(y_gt)

        test_losses.append(test_loss / total_batch)
        test_accs.append(correct / total)
        print(f'test_loss: {test_losses[-1]: .4f}, test_acc: {test_accs[-1]*100: 2f}')
        
    if test_accs[-1] > best_test_acc:
        best_test_acc = test_accs[-1]

In [ ]:
best_test_acc_epoch = np.array(test_accs).argmax() + 1
print('Best test accuracy {:.2f}% in epoch {}.'.format(best_test_acc*100, best_test_acc_epoch))

In [ ]:
df = pd.DataFrame({  # save model training process into csv file
        'loss': train_losses,
        'test_loss': test_losses,
        'acc': train_accs,
        'test_acc': test_accs
    })
df.to_csv(os.path.join('csv', 'loss_and_acc.csv'))

In [ ]:
df = pd.read_csv(os.path.join('csv', 'loss_and_acc.csv'))
test_accs = df['test_acc']
train_accs = df['acc']
test_losses = df['test_loss']
train_losses = df['loss']


fig, ax = plt.subplots(1, 2, figsize=(15, 4))

ax[0].plot(train_losses, color='red')
ax[0].plot(test_losses, color='green')
ax[0].set_xticks([0, 100, 200, 300, 400])
ax[0].set_xlabel('Epoch', size=16)
ax[0].set_ylabel('Loss', size=16)
ax[0].set_ylim(0.9, 1.1)
ax[0].set_yticks([0.9, 1.0, 1.1, 1.2])
ax[0].grid(alpha=0.5)
ax[0].tick_params(labelsize=16)
ax[0].legend(['Train', 'Test'], loc='right', fontsize=16)


ax[1].set_ylim(0.7, 1.0)
ax[1].set_yticks([0.7, 0.8, 0.9, 1.0])
ax[1].plot(train_accs, color='red')
ax[1].plot(test_accs, color='green')
yt = ax[1].get_yticks()
ax[1].set_yticklabels(['{:,.0%}'.format(x) for x in yt])
ax[1].set_xticks([0, 100, 200, 300, 400])
ax[1].set_xlabel('Epoch', size=16)
ax[1].set_ylabel('Accuracy', size=16, labelpad=-5)
ax[1].grid(alpha=0.5)
ax[1].tick_params(labelsize=16)
ax[1].legend(['Train', 'Test'], loc='right', fontsize=16)

fig.savefig('loss_acc_conv2_split_seed.pdf', bbox_inches='tight')

In [22]:
duet.close()